In [1]:
from datetime import datetime as dt

from src.shared import Guild, Channel, ChannelType, Message, User, login_utils
from src.server import db, asset_generator

In [2]:
# dangerous!!!
for collection in db.db.list_collection_names():
    db.db.drop_collection(collection)

In [3]:
db.messages.create_index("channel_id")
db.users.create_index("username", unique=True)
db.channels.create_index("guild_id")
db.guilds.create_index("join_code", unique=True)

'join_code_1'

In [4]:
u = asset_generator.generate_user("test", login_utils.hash_password("Pass1234"))

In [5]:
guilds = [asset_generator.generate_guild(f"Test guild {i}", u.id) for i in range(5)]


In [6]:
u2 = asset_generator.generate_user("amogus", login_utils.hash_password("Amogus123"))

In [7]:
for guild in guilds:
    db.user_join_guild(u2.id, guild.id)

In [8]:
for guild in guilds:
    for i in range(5):
        asset_generator.generate_channel(f"test-channel-{i}", ChannelType.TEXT, guild.id)

In [9]:
for channel_db in db.channels.find():
    channel: Channel = Channel.from_db_dict(channel_db)
    buffer: list[dict] = []
    for i in range(150):
        id = asset_generator.get_id()
        d = Message(id, channel.id, f"test message {i} in channel {channel.name} in guild {channel.guild_id}", attachment=None, author=u, timestamp=id >> asset_generator.TAG_LENGTH).to_db_dict()
        d["author_id"] = u.id
        del d["author"]
        d["attachment_id"] = None
        del d["attachment"]
        buffer.insert(0, d)
    db.messages.insert_many(buffer)


In [10]:
# Generating a test image

from src.shared.attachment import AttachmentType


with open("./assets/sus.png", "rb") as f:
    file = f.read()
a = asset_generator.generate_attachment(file, AttachmentType.IMAGE, "sus.png")

# Adding to a channel
channel = db.channels.find_one()
assert channel is not None
channel: Channel = Channel.from_db_dict(channel)

m = asset_generator.generate_message(channel.id, "test message with image", u, a)



In [11]:
db.get_messages(channel.id, 0, 10)

[Message(id=112389884743367, channel_id=112389884739591, content='test message with image', attachment=Attachment(id=112389884743366, filename='sus.png', a_type=<AttachmentType.IMAGE: 'image'>, width=842, height=1024, size=21624), author=User(id=112389884674048, username='test'), timestamp=1714933544),
 Message(id=112389884739765, channel_id=112389884739591, content='test message 149 in channel test-channel-0 in guild 112389884674049', attachment=None, author=User(id=112389884674048, username='test'), timestamp=1714933544),
 Message(id=112389884739764, channel_id=112389884739591, content='test message 148 in channel test-channel-0 in guild 112389884674049', attachment=None, author=User(id=112389884674048, username='test'), timestamp=1714933544),
 Message(id=112389884739763, channel_id=112389884739591, content='test message 147 in channel test-channel-0 in guild 112389884674049', attachment=None, author=User(id=112389884674048, username='test'), timestamp=1714933544),
 Message(id=112389